In [1]:
import numpy as np
import pandas as pd

### Import dataset from Academic Paper

In [2]:
# Read original dataset from academic paper (from Github)
original_df = pd.read_csv('Original_data.csv')

In [3]:
# Create a column that combines GVKEY with Year
original_df['gvkey_year'] = original_df['gvkey'].astype(str) + '_' + original_df['fyear'].astype(str)

In [4]:
# Store all GVKEYs in a list
gvkey_list = original_df['gvkey'].to_list()
len(gvkey_list)

146045

In [5]:
# Keep only unique GVKEYs
gvkey_set = set(gvkey_list)
len(gvkey_set)

18444

### Store GVKEY's in a text file to upload to WRDS database

In [6]:
# Define the output file path
output_file = 'gvkey_list.txt'

# Write gvkey_set to the output file
with open(output_file, 'w') as f:
    for gvkey in gvkey_set:
        f.write(str(gvkey) + '\n')

### Import dataset from WRDS for credit ratings

In [7]:
# Read credit ratings file from WRDS
ratings_df = pd.read_csv('wrds_credit_ratings.csv')

C:\Users\footb\AppData\Local\Temp\ipykernel_11500\3415646772.py:2: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings_df = pd.read_csv('wrds_credit_ratings.csv')


In [8]:
# See the types of credit ratings in the SPLTICRM column
ratings_df['splticrm'].unique()

array(['BBB', 'BBB-', 'BB-', 'BB', 'BB+', nan, 'B+', 'B', 'CCC+', 'B-',
       'A', 'A-', 'BBB+', 'CCC', 'D', 'AA+', 'AAA', 'AA', 'A+', 'CCC-',
       'AA-', 'SD', 'CC', 'N.M.', 'C'], dtype=object)

In [9]:
rating_mapping = {
    'AAA': 1, 'AA+': 2, 'AA': 3, 'AA-': 4,
    'A+': 5, 'A': 6, 'A-': 7,
    'BBB+': 8, 'BBB': 9, 'BBB-': 10,
    'BB+': 11, 'BB': 12, 'BB-': 13,
    'B+': 14, 'B': 15, 'B-': 16,
    'CCC+': 17, 'CCC': 18, 'CCC-': 19,
    'CC': 20, 'C': 21, 'SD': 22,
    'D': 23, 'N.M.': 24
}

In [10]:
# Function to apply the mapping and convert ratings to ordinal rankings
def convert_to_ordinal(rating):
    return rating_mapping.get(rating, None)

# Apply the function to create a new column with ordinal rankings
ratings_df['ordinal_ranking'] = ratings_df['splticrm'].apply(convert_to_ordinal)

In [11]:
# To get annual ratings of a company, we use the start of year rating (Jan 31)

# Convert 'datadate' to datetime format
ratings_df['datadate'] = pd.to_datetime(ratings_df['datadate'])

# Filter ratings_df to keep only January 31 values
ratings_annual_df = ratings_df[ratings_df['datadate'].dt.month == 1].copy()

# Create a year column
ratings_annual_df['year']=ratings_annual_df['datadate'].dt.year

# Create a column that combines GVKEY with Year
ratings_annual_df['gvkey_year'] = ratings_annual_df['gvkey'].astype(str) + '_' + ratings_annual_df['year'].astype(str)

In [12]:
# Sort the DataFrame by gvkey and year
ratings_annual_df.sort_values(by=['gvkey', 'year'], inplace=True)

# Calculate the change in ordinal ranking
ratings_annual_df['ordinal_ranking_change'] = ratings_annual_df.groupby('gvkey')['ordinal_ranking'].diff()

In [13]:
ratings_annual_df.head(10)

,gvkey,splticrm,spsdrm,spsticrm,datadate,cik,fyrc,ggroup,gind,gsector,...,spcindcd,spcseccd,spcsrc,conm,tic,cusip,ordinal_ranking,year,gvkey_year,ordinal_ranking_change
0,1004,BBB,NaN,NaN,1989-01-31,1750.0,5,2010.0,201010.0,20.0,...,110.0,925.0,B,AAR CORP,AIR,000361105,9.0,1989,1004_1989,NaN
12,1004,BBB,NaN,NaN,1990-01-31,1750.0,5,2010.0,201010.0,20.0,...,110.0,925.0,B,AAR CORP,AIR,000361105,9.0,1990,1004_1990,0.0
24,1004,BBB,NaN,NaN,1991-01-31,1750.0,5,2010.0,201010.0,20.0,...,110.0,925.0,B,AAR CORP,AIR,000361105,9.0,1991,1004_1991,0.0
36,1004,BBB,NaN,NaN,1992-01-31,1750.0,5,2010.0,201010.0,20.0,...,110.0,925.0,B,AAR CORP,AIR,000361105,9.0,1992,1004_1992,0.0
48,1004,BBB,NaN,NaN,1993-01-31,1750.0,5,2010.0,201010.0,20.0,...,110.0,925.0,B,AAR CORP,AIR,000361105,9.0,1993,1004_1993,0.0
60,1004,BBB,NaN,NaN,1994-01-31,1750.0,5,2010.0,201010.0,20.0,...,110.0,925.0,B,AAR CORP,AIR,000361105,9.0,1994,1004_1994,0.0
72,1004,BBB-,NaN,NaN,1995-01-31,1750.0,5,2010.0,201010.0,20.0,...,110.0,925.0,B,AAR CORP,AIR,000361105,10.0,1995,1004_1995,1.0
84,1004,BBB-,NaN,NaN,1996-01-31,1750.0,5,2010.0,201010.0,20.0,...,110.0,925.0,B,AAR CORP,AIR,000361105,10.0,1996,1004_1996,0.0
96,1004,BBB-,NaN,NaN,1997-01-31,1750.0,5,2010.0,201010.0,20.0,...,110.0,925.0,B,AAR CORP,AIR,000361105,10.0,1997,1004_1997,0.0
108,1004,BBB,NaN,NaN,1998-01-31,1750.0,5,2010.0,201010.0,20.0,...,110.0,925.0,B,AAR CORP,AIR,000361105,9.0,1998,1004_1998,-1.0


### Steps for merging dataset from Academic Paper with credit ratings data

In [14]:
# Merge original_df with ratings_annual_df based on 'gvkey_year'
merged_df = pd.merge(original_df, ratings_annual_df, on='gvkey_year', how='left')
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146045 entries, 0 to 146044
Data columns (total 69 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   fyear                   146045 non-null  int64         
 1   gvkey_x                 146045 non-null  int64         
 2   p_aaer                  964 non-null     float64       
 3   misstate                146045 non-null  int64         
 4   act                     146045 non-null  float64       
 5   ap                      146045 non-null  float64       
 6   at                      146045 non-null  float64       
 7   ceq                     146045 non-null  float64       
 8   che                     146045 non-null  float64       
 9   cogs                    146045 non-null  float64       
 10  csho                    146045 non-null  float64       
 11  dlc                     146045 non-null  float64       
 12  dltis                   146045

In [15]:
# Keep only the rows that have a value for the change in ordinal ranking
merged_df2 = merged_df.dropna(subset=['ordinal_ranking_change'])
merged_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23141 entries, 8 to 146034
Data columns (total 69 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   fyear                   23141 non-null  int64         
 1   gvkey_x                 23141 non-null  int64         
 2   p_aaer                  266 non-null    float64       
 3   misstate                23141 non-null  int64         
 4   act                     23141 non-null  float64       
 5   ap                      23141 non-null  float64       
 6   at                      23141 non-null  float64       
 7   ceq                     23141 non-null  float64       
 8   che                     23141 non-null  float64       
 9   cogs                    23141 non-null  float64       
 10  csho                    23141 non-null  float64       
 11  dlc                     23141 non-null  float64       
 12  dltis                   23141 non-null  float

In [16]:
merged_df2[merged_df2['misstate'] == 1].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 266 entries, 1662 to 140297
Data columns (total 69 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   fyear                   266 non-null    int64         
 1   gvkey_x                 266 non-null    int64         
 2   p_aaer                  266 non-null    float64       
 3   misstate                266 non-null    int64         
 4   act                     266 non-null    float64       
 5   ap                      266 non-null    float64       
 6   at                      266 non-null    float64       
 7   ceq                     266 non-null    float64       
 8   che                     266 non-null    float64       
 9   cogs                    266 non-null    float64       
 10  csho                    266 non-null    float64       
 11  dlc                     266 non-null    float64       
 12  dltis                   266 non-null    floa

In [17]:
# Drop unnecessary columns
drop_columns = ['gvkey_y','spsdrm','spsticrm','spcindcd','spcsrc']
merged_df3 = merged_df2.drop(columns=drop_columns)
merged_df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23141 entries, 8 to 146034
Data columns (total 64 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   fyear                   23141 non-null  int64         
 1   gvkey_x                 23141 non-null  int64         
 2   p_aaer                  266 non-null    float64       
 3   misstate                23141 non-null  int64         
 4   act                     23141 non-null  float64       
 5   ap                      23141 non-null  float64       
 6   at                      23141 non-null  float64       
 7   ceq                     23141 non-null  float64       
 8   che                     23141 non-null  float64       
 9   cogs                    23141 non-null  float64       
 10  csho                    23141 non-null  float64       
 11  dlc                     23141 non-null  float64       
 12  dltis                   23141 non-null  float

In [18]:
# Separate p_aaer variable from merged_df4 to remove NaN values from other variables
p_aaer_df = merged_df3[['gvkey_year','p_aaer']]
merged_df4 = merged_df3.drop(columns='p_aaer')
merged_df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23141 entries, 8 to 146034
Data columns (total 63 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   fyear                   23141 non-null  int64         
 1   gvkey_x                 23141 non-null  int64         
 2   misstate                23141 non-null  int64         
 3   act                     23141 non-null  float64       
 4   ap                      23141 non-null  float64       
 5   at                      23141 non-null  float64       
 6   ceq                     23141 non-null  float64       
 7   che                     23141 non-null  float64       
 8   cogs                    23141 non-null  float64       
 9   csho                    23141 non-null  float64       
 10  dlc                     23141 non-null  float64       
 11  dltis                   23141 non-null  float64       
 12  dltt                    23141 non-null  float

In [19]:
# Drop NaN
merged_df5 = merged_df4.dropna()
merged_df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21393 entries, 8 to 145929
Data columns (total 63 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   fyear                   21393 non-null  int64         
 1   gvkey_x                 21393 non-null  int64         
 2   misstate                21393 non-null  int64         
 3   act                     21393 non-null  float64       
 4   ap                      21393 non-null  float64       
 5   at                      21393 non-null  float64       
 6   ceq                     21393 non-null  float64       
 7   che                     21393 non-null  float64       
 8   cogs                    21393 non-null  float64       
 9   csho                    21393 non-null  float64       
 10  dlc                     21393 non-null  float64       
 11  dltis                   21393 non-null  float64       
 12  dltt                    21393 non-null  float

In [20]:
merged_df5[merged_df5['misstate'] == 1].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 258 entries, 1662 to 137754
Data columns (total 63 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   fyear                   258 non-null    int64         
 1   gvkey_x                 258 non-null    int64         
 2   misstate                258 non-null    int64         
 3   act                     258 non-null    float64       
 4   ap                      258 non-null    float64       
 5   at                      258 non-null    float64       
 6   ceq                     258 non-null    float64       
 7   che                     258 non-null    float64       
 8   cogs                    258 non-null    float64       
 9   csho                    258 non-null    float64       
 10  dlc                     258 non-null    float64       
 11  dltis                   258 non-null    float64       
 12  dltt                    258 non-null    floa

In [21]:
# Re-merge the p_aaer variable with merged_df5
merged_df6 = pd.merge(merged_df5, p_aaer_df, on='gvkey_year', how='left')
merged_df6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21393 entries, 0 to 21392
Data columns (total 64 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   fyear                   21393 non-null  int64         
 1   gvkey_x                 21393 non-null  int64         
 2   misstate                21393 non-null  int64         
 3   act                     21393 non-null  float64       
 4   ap                      21393 non-null  float64       
 5   at                      21393 non-null  float64       
 6   ceq                     21393 non-null  float64       
 7   che                     21393 non-null  float64       
 8   cogs                    21393 non-null  float64       
 9   csho                    21393 non-null  float64       
 10  dlc                     21393 non-null  float64       
 11  dltis                   21393 non-null  float64       
 12  dltt                    21393 non-null  float6

### Export combined dataset (original and credit ratings) to a CSV file

In [22]:
merged_df6.to_csv('FilteredDataJuly19.csv')